In [26]:
print("Language classifier")

Language classifier


In [41]:
import pandas as pd

In [64]:
data = pd.read_csv("data.csv")
test_data = pd.read_csv("test.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  500 non-null    object
 1   fingers  500 non-null    int64 
 2   tail     500 non-null    object
 3   species  500 non-null    object
dtypes: int64(1), object(3)
memory usage: 15.8+ KB


In [65]:
data

,message,fingers,tail,species
0,pluvia arbor aquos,4,no,Aquari
1,cosmix xeno nebuz odbitaz,5,yes,Zorblax
2,solarix glixx novum galaxum quasar,5,yes,Zorblax
3,arbor insectus pesros ekos dootix nimbus,2,yes,Florian
4,mermax drakos lorix epikoz deftax,4,no,Faerix
...,...,...,...,...
495,empathix sadix disgux dredax pridius afgstix e...,2,no,Emotivor
496,quasar ustron nebulax meteorn,4,no,Quixnar
497,astron xeno ceaestar astron kometa,6,yes,Zorblax
498,sporzom nimbus terram terranix aviana ekos nimbub,2,yes,Florian


In [66]:
data['tail']=data['tail'].map({'yes':1, 'no':0})

In [67]:
data

,message,fingers,tail,species
0,pluvia arbor aquos,4,0,Aquari
1,cosmix xeno nebuz odbitaz,5,1,Zorblax
2,solarix glixx novum galaxum quasar,5,1,Zorblax
3,arbor insectus pesros ekos dootix nimbus,2,1,Florian
4,mermax drakos lorix epikoz deftax,4,0,Faerix
...,...,...,...,...
495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor
496,quasar ustron nebulax meteorn,4,0,Quixnar
497,astron xeno ceaestar astron kometa,6,1,Zorblax
498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian


In [68]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['species_number'] = label_encoder.fit_transform(data['species'])

In [69]:
data

,message,fingers,tail,species,species_number
0,pluvia arbor aquos,4,0,Aquari,0
1,cosmix xeno nebuz odbitaz,5,1,Zorblax,9
2,solarix glixx novum galaxum quasar,5,1,Zorblax,9
3,arbor insectus pesros ekos dootix nimbus,2,1,Florian,4
4,mermax drakos lorix epikoz deftax,4,0,Faerix,3
...,...,...,...,...,...
495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor,2
496,quasar ustron nebulax meteorn,4,0,Quixnar,7
497,astron xeno ceaestar astron kometa,6,1,Zorblax,9
498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian,4


In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_message = vectorizer.fit_transform(data['message'])

In [71]:
X_message.shape

(500, 636)

In [72]:
X_message[:5].todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [73]:
feature_names = vectorizer.get_feature_names_out()
print(feature_names[:40]) 

['acon' 'aeon' 'aeop' 'aeqn' 'afgstix' 'airenix' 'akiana' 'akon' 'algorid'
 'algorix' 'aneius' 'angsqix' 'angstic' 'angstix' 'angstox' 'angxtix'
 'annstix' 'anxids' 'anxims' 'anxius' 'anxiuz' 'anzius' 'aoblax' 'aqjos'
 'aqpos' 'aquoe' 'aquon' 'aquos' 'arbon' 'arbor' 'arbsr' 'arcor' 'ariana'
 'asbron' 'aslron' 'astrln' 'astrob' 'astroh' 'astron' 'astrrn']


In [74]:
first_message_tfidf = X_message[0].todense().tolist()[0] 
word_tfidf_pairs = [(feature_names[i], first_message_tfidf[i]) for i in range(len(feature_names)) if first_message_tfidf[i] > 0]
print(word_tfidf_pairs)

[('aquos', 0.5994832620321487), ('arbor', 0.582840857925647), ('pluvia', 0.5485584315966621)]


In [75]:
sparsity = 1.0 - (X_message.count_nonzero() / float(X_message.shape[0] * X_message.shape[1]))
print(f"Sparsity: {sparsity * 100:.2f}%")

Sparsity: 99.19%


In [76]:
import numpy as np
from scipy.sparse import hstack

X_features = hstack([X_message, np.array(data[['fingers', 'tail']])])
y = data['species']

In [77]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.1, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

Validation Accuracy: 0.88


In [78]:
test_data['tail']=test_data['tail'].map({'yes':1, 'no':0})

In [81]:
test_data

,message,fingers,tail
0,iephyr terram nimbus terram faunar foliar,2,0
1,joyzor uleex luvium caloox shockus blissae,4,1
2,aquos arbor ventuc,4,1
3,nympha nympha epikoz nympha mythox mythox mythox,3,0
4,diitax sibenix fabulon,4,1
...,...,...,...
294,fabuion drakos lorix relikum cyclopix,4,1
295,codex cybrex algorix synapz mechan nanozom dotax,5,1
296,centarex mythox nympha krakos,3,1
297,orbitaz astron glixx novara novrm ufox qcasar ...,3,0


In [82]:
X_test_message = vectorizer.transform(test_data['message'])
X_test_features = hstack([X_test_message, np.array(test_data[['fingers', 'tail']])])

In [83]:
y_test_pred = model.predict(X_test_features)

In [88]:
y_test_pred

array(['Aquari', 'Sentire', 'Florian', 'Mythron', 'Mythron', 'Faerix',
       'Faerix', 'Faerix', 'Emotivor', 'Florian', 'Mythron', 'Emotivor',
       'Zorblax', 'Quixnar', 'Mythron', 'Aquari', 'Florian', 'Cybex',
       'Aquari', 'Aquari', 'Quixnar', 'Emotivor', 'Quixnar', 'Nexoon',
       'Mythron', 'Nexoon', 'Aquari', 'Sentire', 'Emotivor', 'Mythron',
       'Cybex', 'Emotivor', 'Mythron', 'Emotivor', 'Mythron', 'Emotivor',
       'Aquari', 'Florian', 'Aquari', 'Quixnar', 'Emotivor', 'Quixnar',
       'Mythron', 'Nexoon', 'Aquari', 'Emotivor', 'Quixnar', 'Quixnar',
       'Zorblax', 'Florian', 'Emotivor', 'Quixnar', 'Sentire', 'Quixnar',
       'Emotivor', 'Emotivor', 'Sentire', 'Emotivor', 'Mythron', 'Cybex',
       'Cybex', 'Cybex', 'Mythron', 'Emotivor', 'Sentire', 'Mythron',
       'Sentire', 'Mythron', 'Nexoon', 'Quixnar', 'Quixnar', 'Emotivor',
       'Emotivor', 'Zorblax', 'Emotivor', 'Sentire', 'Cybex', 'Faerix',
       'Nexoon', 'Emotivor', 'Emotivor', 'Cybex', 'Emotivor', 

In [93]:
pd.DataFrame(y_test_pred)

,0
0,Aquari
1,Sentire
2,Florian
3,Mythron
4,Mythron
...,...
294,Mythron
295,Nexoon
296,Mythron
297,Quixnar
